In [3]:
import sys
import numpy as np
# import hipsternet.neuralnet as nn
import impl.RNN as rnn
import impl.solver as solver

In [4]:
time_step = 10
n_iter = 100000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [5]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [6]:
vocab_size = len(char_to_idx)

In [7]:
# net = nn.LSTM(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
net = rnn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.GRU(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [ ]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.4720
 aailrog dos fan liEotei ld 0hn srmciicl kAt-to lauoem cnd ans-arp nd antd tod Jrrd au,b xsmsSia,gar


Iter-2000 loss: 2.8577
ord unt  mule ta  cespan ookth iosth vn chiod Tirts 
nnd lhec an nan ieac on Jt5t wfid m thd iUetaat


Iter-3000 loss: 2.4522
rinisen pithe U%pin th  eithela ins, lovg t8inamtint ithe Toutirund on tied in tho an in . i本 parcin


Iter-4000 loss: 2.1820
udevt ipal tur of the pentoranify  pftury -nd thecethe wo1st -yApourNsjo7, Kd kurll co rory, Whalang


Iter-5000 loss: 1.9871
cof8gcil of the Gorad Aoved Ny 19zB-lard in che fered bipat -nat cosithe Gar io eritGdy8Est ex4the t


Iter-6000 loss: 1.8397
orlo-lawa ing mules fand iing raliun en miclly d reg lar os chr dirleite lerit nt orel as. uf p w9ol


Iter-7000 loss: 1.6964
ate in 18,2, min tretoky the cauler, in litg of ingter Efpure f
rhe ia ty s.y Ruan. Tfere it enasiry


Iter-8000 loss: 1.5977
ily aricston inth-dupenie, chined 3w9.1land who Arthe worrtPHones. Japan 's Frokth an coullyy. 

Iter-25000 loss: 0.9607
rins whal morol ded fored War Isto Asiand of the SIl bhe Panis pe tare ma nsulatirctarich od the fou


Iter-26000 loss: 0.9271
 f unt of ifteona and 1agest ese hestory find itrl the Rity, which the rored ihe wonseicty in panise


Iter-27000 loss: 0.9230
hoby, which inslargest ncobomontl conre tienter and Thi nasiop ofrel whichurlargest prorgeolly siage


Iter-28000 loss: 0.8909
ve in the Global Pal Cos andechas hastare is ha Sta 12t ry bary cand if lo the eally. Arlation ef to


Iter-29000 loss: 0.8842
igrlothech fits intirgal perrias. fxuriag, Japan was natal mior, Hand theake Koulande, u7. Thilation


Iter-30000 loss: 0.8647
f and rasel. Toklolaticy buntorbes mares ingirl. Ind. Asth ussired an with rand malited as ofeldogea


Iter-31000 loss: 0.8631
aliled Sceceand Rursovilcias anded lorled in oJag85Jail 1Dth largest rreepered Japanef Japanest and 


Iter-32000 loss: 0.8285
kJapho lleton-no sslf12d loughtref Japan hos anthe fits and hoslompic inty by ts allite

Iter-49000 loss: 0.6537
 the East of the Rising Sun erches of thel ar and malicatincabiving pumbeankes sha Nowsther 35 millo


Iter-50000 loss: 0.6311
s a. Japana and Worto the Global Polithe Scand it int int ea ionat roled ind Napiod ofsiro Is rhesed


Iter-51000 loss: 0.6250
ed milited in irchaicha wish ia the eate liGdea of ty tooLokulion to warch ca ind consth- argheste F


Iter-52000 loss: 0.6139
ty. Ind. Japan to expand andesored the the Seand canctha and innto 4a, KduJapan's and Napmen citates


Iter-53000 loss: 0.6174
an eact on the world's ko anutsofliond Nipses of the Uan th to the G20 of mean ston miand. Japanes t


Iter-54000 loss: 0.6140
my by by buthe vighest regite lntite in the Emvlliglle Enpure 2045––J:parigofand of is larges the Fi


Iter-55000 loss: 0.6111
wr areessivelirc the Upchixth dhith roftury fwis the pithisatire molalard iton is the cores alobith.


Iter-56000 loss: 0.5856
e eopur1868-no Asian, imeas opurd fitht Siggol wire ery Wev loxto ten aighaca mofala as

In [ ]:
=========================================================================
Iter-100000 loss: 0.3913
=========================================================================
the 1pman a inusy rotory surrowho tOkike upporter wor peven des The easty porterized itivecend perce
=========================================================================


Out[7]:
<hipsternet.neuralnet.RNN at 0x7fc2f5182e80>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1615
=========================================================================
the name of the 12th century until 1868, Japan was ruled by successime Wer of 1941, which country is
=========================================================================


Out[16]:
<hipsternet.neuralnet.LSTM at 0x7f67fe0c9978>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1333
=========================================================================
the Coun resilate a in the population. Russided in the OECD and the world's fourth-largest econompic
=========================================================================


Out[10]:
<hipsternet.neuralnet.GRU at 0x7fc2cec1e1d0>